In [ ]:
import sympy as sym
sym.init_printing()
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.interpolate import InterpolatedUnivariateSpline

In [ ]:
plt.style.use('./my_style.py')
import matplotlib.ticker as mticker

from cycler import cycler
import matplotlib.cm     as cm

# Example use of solver 

In [ ]:
from solver import KmouExpansionJordan

In [ ]:
# initilise solver
dum_kmou = KmouExpansionJordan(beta=0.2, K0=1, lamb_val=1.476)

In [ ]:
# dum_kmou.tune_lambda(E_target=1.105861, a_target=0.875543)
dum_kmou.tune_lambda(E_target=1, a_target=1, frame='Einstein', maxiter=10) # tune lambda to recover E=E_target in the desired frame
# dum_kmou.solve() # to compute solutions without tuning

In [ ]:
dum_kmou.lamb_val

In [ ]:
# create dataframe with some output quantities
header, table = dum_kmou.eval()
df_out = pd.DataFrame(np.array(table).T, columns=header).set_index('a')

In [ ]:
df_out

In [ ]:
# Check E'/E
plt.semilogx(df_out.E_kmou_a*df_out.index/df_out.E_kmou)
plt.hlines(-3/2, 1e-3, 1, colors='k', linestyles='--')

### Comparison with Ben: $K_0=1$, $\beta=0.2$, $n=2$

In [ ]:
header_str = 'a,  H_MG,  H_LCDM, H_MG/H_LCDM,  aH dH/da / H0^2 ,  phi , d phi/ dlna'
header = [s.strip(' ')for s in header_str.split(',')]

df_Ben_BG = pd.DataFrame(np.loadtxt('./Data/Background/background_k01_glam.dat'), columns=header).set_index('a')

In [ ]:
df_Ben_BG['A'] = np.e**(df_Ben_BG.phi*0.2)
df_Ben_BG['a_J'] = df_Ben_BG['A']*df_Ben_BG.index

In [ ]:
a_E = np.logspace(-4,0,1000)
a_J = dum_kmou.get_a_Jor(a_E)

In [ ]:
header, table = dum_kmou.eval(a_vals=a_J)
df_out = pd.DataFrame(np.array(table).T, columns=header).set_index('a')

In [ ]:
plt.semilogx((df_out['E_kmou']/dum_kmou.E_LCDM_fun(a_E)))
plt.semilogx((df_Ben_BG['H_MG']/df_Ben_BG['H_LCDM']),'--')
# plt.ylim(0.9,1.1)

# Compute growth factor

In [ ]:
dum_kmou.get_growth()

In [ ]:
dum_kmou.get_growth_LCDM()

In [ ]:
plt.plot(dum_kmou.D_kmou_J['t'],dum_kmou.D_kmou_J['y'][1])
plt.plot(dum_kmou.D_LCDM['t'],dum_kmou.D_LCDM['y'][1])


# Compare with old results

In [ ]:
old_header = ['a_E', 'a_J', 'E_J', 'dE_J_da', 'phi_J', 'dphi_J_dtau', 'G_eff_G', 'A_conf', 'D_kmou']
df_old = pd.DataFrame(np.loadtxt('./Data/Output/n2-beta0p2-k1/out_table.txt'), columns=old_header)

In [ ]:
ngt2_header = ['a_E', 'a_J', 'E_J', 'dE_J_da', 'phi_J', 'dphi_J_dtau', 'G_eff_G', 'A_conf', 'D_kmou']
df_ngt2 = pd.DataFrame(np.loadtxt('/Users/bartolomeo/Downloads/out_table.txt'), columns=ngt2_header)

In [ ]:
header, table = dum_kmou.eval(a_vals=df_old['a_J'])
df_out = pd.DataFrame(np.array(table).T, columns=header).set_index('a')

In [ ]:
header, table = dum_kmou.eval(a_vals=df_ngt2['a_J'])
df_out_ngt2 = pd.DataFrame(np.array(table).T, columns=header).set_index('a')

In [ ]:
# produce familiar growth ratio with LCDM
a_E = np.logspace(-4,0,1000)
a_J = dum_kmou.get_a_Jor(a_E)

plt.semilogx(a_E , dum_kmou.D_kmou_J.sol(a_J)[1]/dum_kmou.D_LCDM.sol(a_E)[1])
plt.semilogx(df_old.a_E , df_old['D_kmou']/dum_kmou.D_LCDM.sol(df_old.a_E)[1])
plt.semilogx(df_ngt2.a_E , df_ngt2['D_kmou']/dum_kmou.D_LCDM.sol(df_ngt2.a_E)[1])


In [ ]:
plt.plot(df_out['E_kmou']/df_old['E_J'].values)
plt.plot(df_out['E_kmou_a']/df_old['dE_J_da'].values)
plt.plot(df_out['phi']/df_old['phi_J'].values)
plt.ylim(0.99,1.01)

In [ ]:
plt.plot(df_out_ngt2['E_kmou']/df_ngt2['E_J'].values)
plt.plot(df_out_ngt2['E_kmou_a']/df_ngt2['dE_J_da'].values)
plt.plot(df_out_ngt2['phi']/df_ngt2['phi_J'].values)
plt.ylim(0.99,1.01)

In [ ]:
plt.semilogx(df_out.index**2*df_out.E_kmou*df_out['phi_a']/df_old['dphi_J_dtau'].values*1/2997)
plt.semilogx(df_out_ngt2.index**2*df_out_ngt2.E_kmou*df_out_ngt2['phi_a']/df_ngt2['dphi_J_dtau'].values,'--')
plt.ylim(0.99,1.01)

In [ ]:
df_out

In [ ]:
plt.plot(InterpolatedUnivariateSpline(df_out.index, df_out['E_kmou']).derivative()(df_out.index)/
        df_out['E_kmou_a'])
plt.plot(InterpolatedUnivariateSpline(df_out_ngt2.index, df_out_ngt2['E_kmou']).derivative()(df_out_ngt2.index)/
        df_out_ngt2['E_kmou_a'])
plt.plot(df_ngt2.a_J, InterpolatedUnivariateSpline(df_ngt2.a_J, df_ngt2.E_J).derivative()(df_ngt2.a_J)/
        df_ngt2['dE_J_da'])
plt.ylim(0.99,1.01)